In [1]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2
import json

import gmaps
import gmaps.geojson_geometries


from geographiclib.geodesic import Geodesic

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [5]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [6]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    


In [7]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [8]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")
    print(df.head(10)) 

In [9]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [10]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [11]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [12]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [13]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    print("\n-------------------------------------------------------------------------------")
    print("  Zip Codes within " + str(miles) + " mile(s) of " + station + " BART Station")
    print("-------------------------------------------------------------------------------\n")
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        print("     zip:", zip, "  population: ", f'{population:10,}')
        total_population += population
        
    
    print("\n-------------------------------------------------------------------------------")
    print("  Total Population: ", f'{total_population:10,}')
    print("-------------------------------------------------------------------------------")

In [14]:
def my_get_node_list():
    "get a list of nodes in the current graph"
    
    query = "match (n) return n.name as name"
    
    result = session.run(query)
    
    node_list = []
    
    for r in result:
        node_list.append(r["name"])
        
    node_list = sorted(node_list)
    
    return node_list

In [15]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [16]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [17]:
cursor = connection.cursor()

In [18]:
my_neo4j_wipe_out_database()

In [19]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------
Empty DataFrame
Columns: [node_name_1, node_1_labels, relationship_type, node_name_2, node_2_labels]
Index: []


In [20]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(station, line_station, 0)

In [21]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 114
  Relationships: 0
-------------------------
Empty DataFrame
Columns: [node_name_1, node_1_labels, relationship_type, node_name_2, node_2_labels]
Index: []


In [22]:
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)

In [23]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 114
  Relationships: 208
-------------------------
                node_name_1 node_1_labels relationship_type  \
0  blue 16th Street Mission     [Station]              LINK   
1  blue 16th Street Mission     [Station]              LINK   
2  blue 16th Street Mission     [Station]              LINK   
3  blue 24th Street Mission     [Station]              LINK   
4  blue 24th Street Mission     [Station]              LINK   
5  blue 24th Street Mission     [Station]              LINK   
6          blue Balboa Park     [Station]              LINK   
7          blue Balboa Park     [Station]              LINK   
8          blue Balboa Park     [Station]              LINK   
9             blue Bay Fair     [Station]              LINK   

                  node_name_2 node_2_labels  
0   green 16th Street Mission     [Station]  
1     red 16th Street Mission     [Station]  
2  yellow 16th Street Mission     [Station]  
3   green 24th Street Mission     [S

In [24]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)

In [25]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 114
  Relationships: 424
-------------------------
                node_name_1 node_1_labels relationship_type  \
0  blue 16th Street Mission     [Station]              LINK   
1  blue 16th Street Mission     [Station]              LINK   
2  blue 16th Street Mission     [Station]              LINK   
3  blue 16th Street Mission     [Station]              LINK   
4  blue 16th Street Mission     [Station]              LINK   
5  blue 24th Street Mission     [Station]              LINK   
6  blue 24th Street Mission     [Station]              LINK   
7  blue 24th Street Mission     [Station]              LINK   
8  blue 24th Street Mission     [Station]              LINK   
9  blue 24th Street Mission     [Station]              LINK   

                  node_name_2 node_2_labels  
0    blue 24th Street Mission     [Station]  
1           blue Civic Center     [Station]  
2   green 16th Street Mission     [Station]  
3     red 16th Street Mission     [S

In [26]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

# Running the degree closeness aglorithm on the Bart Graph
Results will be saved as a data frame then converted to a csv.

In [27]:
query = """

CALL gds.degree.stream('ds_graph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as degree
ORDER BY degree DESC, name

"""

df = my_neo4j_run_query_pandas(query)
df.to_csv('graph_algorithm_closeness_results.csv', index=False)
df

,name,degree
0,blue 16th Street Mission,5.0
1,blue 24th Street Mission,5.0
2,blue Balboa Park,5.0
3,blue Civic Center,5.0
4,blue Coliseum,5.0
...,...,...
109,yellow SFO,2.0
110,yellow Walnut Creek,2.0
111,blue Dublin,1.0
112,gray OAK,1.0


# Droping closeness_table table if exist

In [28]:
connection.rollback()

query = """

drop table if exists closeness_table;

"""

cursor.execute(query)

connection.commit()

# Create and load data into table 

In [29]:
connection.rollback()

query = """

create table closeness_table (
  station varchar(64),
  degree_closeness numeric,
  primary key (station)
);

"""

cursor.execute(query)

connection.commit()

In [30]:
connection.rollback()

query = """

copy closeness_table
from '/user/projects/project-3-drrnl/code/Partner_code/graph_algorithm_closeness_results.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [31]:
query = """

CALL gds.louvain.stream('ds_graph', {includeIntermediateCommunities: false})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

df_community = my_neo4j_run_query_pandas(query)
df_community.to_csv('graph_algorithm_community.csv', index=False)
df_community

,name,community,intermediate_community
0,orange Ashby,42,None
1,orange Downtown Berkeley,42,None
2,orange El Cerrito Plaza,42,None
3,orange El Cerrito del Norte,42,None
4,orange North Berkeley,42,None
...,...,...,...
109,orange Bay Fair,112,None
110,orange Coliseum,112,None
111,orange Fruitvale,112,None
112,orange Lake Merritt,112,None


In [32]:
connection.rollback()

query = """

drop table if exists community_table;

"""

cursor.execute(query)

connection.commit()

In [33]:
connection.rollback()

query = """

create table community_table (
  station varchar(64),
  community int,
  intermediate_community varchar(64),
  primary key (station)
);

"""

cursor.execute(query)

connection.commit()

In [34]:
connection.rollback()

query = """

copy community_table
from '/user/projects/project-3-drrnl/code/Partner_code/graph_algorithm_community.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [35]:
rollback_before_flag = True
rollback_after_flag = True

query = """
SELECT ct.station, ct.degree_closeness, e.exits, com.community
FROM closeness_table as ct
LEFT JOIN exits e ON ct.station LIKE '%' || e.station|| '%'
JOIN community_table as com ON ct.station = com.station
WHERE e.exits IS NOT NULL
ORDER BY com.community DESC, e.exits DESC, ct.degree_closeness DESC;

"""

df_community_final= my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df_community_final.to_csv('graph_algorithm_closeness_exits_community.csv', index=False)
df_community_final

,station,degree_closeness,exits,community
0,green Fruitvale,4,5748,112
1,orange Fruitvale,4,5748,112
2,blue Fruitvale,4,5748,112
3,green Lake Merritt,4,4995,112
4,orange Lake Merritt,4,4995,112
...,...,...,...,...
93,red El Cerrito Plaza,3,3343,42
94,red Richmond,2,3290,42
95,orange Richmond,2,3290,42
96,red North Berkeley,3,2885,42


### Based on the results from the graph_algorithm_closeness_exits_community.csv, it is determined that there are 7 communities and from each community, the station with the most exits and highest closeness coefficient is selected as the pickup location for that community.

In [36]:
pickup_stations = ['Fruitvale', 'MacArthur', 'Fremont', 'Balboa Park', 'Embarcadero', 'Civic Center', 'Downtown Berkeley']

for station in pickup_stations:
    my_station_get_zips(station, 1.2)


-------------------------------------------------------------------------------
  Zip Codes within 1.2 mile(s) of Fruitvale BART Station
-------------------------------------------------------------------------------

     zip: 94601   population:      52,299
     zip: 94606   population:      38,303

-------------------------------------------------------------------------------
  Total Population:      90,602
-------------------------------------------------------------------------------

-------------------------------------------------------------------------------
  Zip Codes within 1.2 mile(s) of MacArthur BART Station
-------------------------------------------------------------------------------

     zip: 94608   population:      30,289
     zip: 94609   population:      22,811

-------------------------------------------------------------------------------
  Total Population:      53,100
-------------------------------------------------------------------------------

-------

### From the list of selected stations in each community, we looked into zip codes that were within 1.2 miles of the stations. We then selected the zip code with the greatest population as the location for the drone pickup station. 


1. Fruitvale pickup: 94601 zip code
2. MacArthur pickup: 94608 zip code
3. Fremont pickup: 94536 zip code
4. Balboa Park pickup: 94112 zip code
5. Embarcadero pickup: 94133 zip code
6. Civic Center pickup: 94109 zip code
7. Downtown Berkeley: 94704 zip code

In [37]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT station, latitude, longitude
FROM stations
WHERE station in ('Fruitvale', 'MacArthur', 'Fremont', 'Balboa Park', 'Embarcadero', 'Civic Center', 'Downtown Berkeley')
ORDER BY station
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,latitude,longitude
0,Balboa Park,37.721667,-122.447500
1,Civic Center,37.779861,-122.413498
2,Downtown Berkeley,37.869799,-122.268197
3,Embarcadero,37.793056,-122.397222
4,Fremont,37.557489,-121.976620
5,Fruitvale,37.774800,-122.224100
6,MacArthur,37.828260,-122.267275


In [38]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT zip, latitude, longitude 
FROM zip_codes
WHERE zip in ('94601', '94608', '94536', '94112', '94133', '94109', '94704')

"""

df_pickup = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df_pickup

,zip,latitude,longitude
0,94109,37.7952,-122.4222
1,94112,37.7203,-122.4430
2,94133,37.8038,-122.4107
3,94536,37.5713,-121.9854
4,94601,37.7767,-122.2184
5,94608,37.8361,-122.2862
6,94704,37.8666,-122.2580


### Determine the time taken for delivery drones to travel from the pickup stations to the nearest BART station, and the time taken for delivery drones to travel from the AGM Berkeley store to the Downtown Berkeley BART station.

The speed of the delivery drone was taken from: https://www.mckinsey.com/~/media/mckinsey/email/rethink/2023/09/2023-09-06b.html#:~:text=Delivery%20drones%20fly%20at%20an,routes%20could%20ease%20traffic%20congestion. 

The source specifies that drones fly at an average speed of 30 to 60 miles per hour. For this project, we used an average drone speed of 45 miles per hour which was converted to 72.4205 km per hour. 

In [39]:
from math import radians, cos, sin, asin, sqrt
def distance(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a)) 
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c * r)

drone_speed = 72.4205  # km/hr

# Fruitvale station to 94601
print('Fruitvale to 94601:', distance(37.774800, 37.7767, -122.224100, -122.2184) / drone_speed * 3600, "s")

# MacArthur station to 94608
print('MacArthur to 94608:', distance(37.828260, 37.8361, -122.267275, -122.2862) / drone_speed  * 3600, "s")

# Fremont station to 94536
print('Fremont to 94536:', distance(37.557489, 37.5713, -121.976620, -121.9854) / drone_speed * 3600, "s")

# Balboa Park station to 94112
print('Balboa Park to 94112:', distance(37.721667, 37.7203, -122.447500, -122.4430) / drone_speed * 3600, "s")

# Embarcadero station to 94133
print('Embarcadero to 94133:', distance(37.793056, 37.8038, -122.397222, -122.4107) / drone_speed * 3600, "s")

# Civic Center station to 94109
print('Civic Center to 94109:', distance(37.779861, 37.7952, -122.413498, -122.4222) / drone_speed * 3600, "s")

# Downtown Berkeley station to 94704
print('Downtown Berkeley to 94704:', distance(37.869799, 37.8666, -122.268197, -122.2580) / drone_speed * 3600, "s")

# Berkeley store to Downtown Berkeley station
print('Berkeley store to Downtown Berkeley station:', distance(37.8555, 37.8666, -122.2604, -122.2580) / drone_speed * 3600, "s")

Fruitvale to 94601: 27.027144623417485 s
MacArthur to 94608: 93.29524640982878 s
Fremont to 94536: 85.48473104947351 s
Balboa Park to 94112: 21.076036842491803 s
Embarcadero to 94133: 83.61916829946293 s
Civic Center to 94109: 92.91719951200685 s
Downtown Berkeley to 94704: 47.87956216487734 s
Berkeley store to Downtown Berkeley station: 62.242361539124424 s


### Create a new csv file containing the travel time of drones between pickup stations and their closest BART stations, and the travel time of drones between the AGM store and the downtown berkeley BART station.

In [40]:
d = {'pickup_zipcode': ['94601', '94601', '94601', '94608', '94608', '94608', '94536', '94536', '94112', '94112', '94112',
                        '94112', '94133', '94133', '94133', '94133', '94109', '94109', '94109', '94109', '94704', '94704',
                       'Berkeley store', 'Berkeley store'], 
    'community_station': ['blue Fruitvale', 'orange Fruitvale', 'green Fruitvale', 'red MacArthur', 'orange MacArthur',
                          'yellow MacArthur', 'orange Fremont', 'green Fremont', 'red Balboa Park', 'yellow Balboa Park',
                          'green Balboa Park', 'blue Balboa Park', 'red Embarcadero', 'yellow Embarcadero',
                          'green Embarcadero', 'blue Embarcadero', 'red Civic Center', 'yellow Civic Center',
                          'green Civic Center', 'blue Civic Center', 'red Downtown Berkeley', 'orange Downtown Berkeley',
                          'red Downtown Berkeley', 'orange Downtown Berkeley'],
    'time': [27, 27, 27, 93, 93, 93, 85, 85, 21, 21, 21, 21, 84, 84, 84, 84, 93, 93, 93, 93, 48, 48, 62, 62]}

df_pickup_to_station = pd.DataFrame(data=d)
df_pickup_to_station.to_csv('pickup_to_station.csv', index=False)

In [41]:
connection.rollback()

query = """

drop table if exists pickup_to_station;

"""

cursor.execute(query)

connection.commit()

In [42]:
connection.rollback()

query = """

create table pickup_to_station (
  pickup_zipcode varchar(80),
  community_station varchar(80),
  time int
);

"""

cursor.execute(query)

connection.commit()

In [43]:
connection.rollback()

query = """

copy pickup_to_station
from '/user/projects/project-3-drrnl/code/Partner_code/pickup_to_station.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [44]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 114
  Relationships: 424
-------------------------
                node_name_1 node_1_labels relationship_type  \
0  blue 16th Street Mission     [Station]              LINK   
1  blue 16th Street Mission     [Station]              LINK   
2  blue 16th Street Mission     [Station]              LINK   
3  blue 16th Street Mission     [Station]              LINK   
4  blue 16th Street Mission     [Station]              LINK   
5  blue 24th Street Mission     [Station]              LINK   
6  blue 24th Street Mission     [Station]              LINK   
7  blue 24th Street Mission     [Station]              LINK   
8  blue 24th Street Mission     [Station]              LINK   
9  blue 24th Street Mission     [Station]              LINK   

                  node_name_2 node_2_labels  
0    blue 24th Street Mission     [Station]  
1           blue Civic Center     [Station]  
2   green 16th Street Mission     [Station]  
3     red 16th Street Mission     [S

In [45]:
connection.rollback()

query = """

SELECT DISTINCT pickup_zipcode 
FROM pickup_to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    pickup_station = row[0]
    my_neo4j_create_node(pickup_station)

In [46]:
connection.rollback()

query = """

SELECT * 
FROM pickup_to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    pickup_station = row[0]
    community_station = row[1]
    travel_time = row[2]
    
    my_neo4j_create_relationship_two_way(pickup_station, community_station, travel_time)

In [47]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 122
  Relationships: 472
-------------------------
  node_name_1 node_1_labels relationship_type          node_name_2  \
0       94109     [Station]              LINK    blue Civic Center   
1       94109     [Station]              LINK   green Civic Center   
2       94109     [Station]              LINK     red Civic Center   
3       94109     [Station]              LINK  yellow Civic Center   
4       94112     [Station]              LINK     blue Balboa Park   
5       94112     [Station]              LINK    green Balboa Park   
6       94112     [Station]              LINK      red Balboa Park   
7       94112     [Station]              LINK   yellow Balboa Park   
8       94133     [Station]              LINK     blue Embarcadero   
9       94133     [Station]              LINK    green Embarcadero   

  node_2_labels  
0     [Station]  
1     [Station]  
2     [Station]  
3     [Station]  
4     [Station]  
5     [Station]  
6     [Station]  

In [48]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

### Total time from store to each pickup location using shortest path algorithm

In [49]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.dijkstra.stream(
    'ds_graph', 
    { sourceNode: source, 
      targetNode: target, 
      relationshipWeightProperty: 'weight'
    }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    gds.util.asNode(sourceNode).name AS from,
    gds.util.asNode(targetNode).name AS to,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
    costs
ORDER BY index

"""

df_time = pd.DataFrame()
source = "Berkeley store"
pickup_locations = ['94601', '94608','94536', '94112', '94133', '94109', '94704']
for loc in pickup_locations:
    target = loc
    row = my_neo4j_run_query_pandas(query, source=source, target=target)
    df_time = df_time.append(row, ignore_index = True)
df_time

,from,to,totalCost,nodes,costs
0,Berkeley store,94601,1289.0,"[Berkeley store, orange Downtown Berkeley, ora...","[0.0, 62.0, 242.0, 482.0, 662.0, 782.0, 962.0,..."
1,Berkeley store,94608,575.0,"[Berkeley store, orange Downtown Berkeley, ora...","[0.0, 62.0, 242.0, 482.0, 575.0]"
2,Berkeley store,94536,3147.0,"[Berkeley store, orange Downtown Berkeley, ora...","[0.0, 62.0, 242.0, 482.0, 662.0, 782.0, 962.0,..."
3,Berkeley store,94112,2363.0,"[Berkeley store, red Downtown Berkeley, red As...","[0.0, 62.0, 242.0, 482.0, 662.0, 782.0, 1082.0..."
4,Berkeley store,94133,1586.0,"[Berkeley store, red Downtown Berkeley, red As...","[0.0, 62.0, 242.0, 482.0, 662.0, 782.0, 1082.0..."
5,Berkeley store,94109,1835.0,"[Berkeley store, red Downtown Berkeley, red As...","[0.0, 62.0, 242.0, 482.0, 662.0, 782.0, 1082.0..."
6,Berkeley store,94704,110.0,"[Berkeley store, orange Downtown Berkeley, 94704]","[0.0, 62.0, 110.0]"


# Data Visualization using Google API

## Connect to Google Maps using your api key;  edit the file gmap_api_key.txt and put in your api key

In [52]:
import googlemaps

In [53]:
f = open('gmap_api_key.txt', 'r')
my_api_key = f.read()
f.close()

gmaps.configure(api_key=my_api_key)

In [54]:
sather_gate_berkeley = (37.870260430419115, -122.25950168579497)
fig = gmaps.figure(center=sather_gate_berkeley, zoom_level=8)

df_markers = df_pickup[['latitude','longitude']]

marker_layer = gmaps.marker_layer(df_markers)
fig.add_layer(marker_layer)



fig

Figure(layout=FigureLayout(height='420px'))

In [63]:
fig = gmaps.figure(center=sather_gate_berkeley, zoom_level=10)

df_markers = df_pickup[['latitude','longitude']]

marker_layer = gmaps.marker_layer(df_markers)
fig.add_layer(marker_layer)

fig.add_layer(gmaps.transit_layer())


fig

Figure(layout=FigureLayout(height='420px'))

In [55]:
departure = (37.793056,-122.397222)
arrival = (37.8038,-122.4107)

gmaps_client = googlemaps.Client(key=my_api_key)

directions = gmaps_client.directions(departure, arrival, mode="driving")

driving_distance = directions[0]['legs'][0]['distance']['text']
print("Driving Distance:", driving_distance)

driving_time = directions[0]['legs'][0]['duration']['text']
print("Driving Time:", driving_time)

fig = gmaps.figure()
driving_route = gmaps.directions_layer(departure, arrival)
fig.add_layer(driving_route)
fig

Driving Distance: 1.5 mi
Driving Time: 10 mins


Figure(layout=FigureLayout(height='420px'))

In [56]:
#
#  Given two points in (latitude, longitude) format, calculate the distance between them in miles
#

def my_calculate_distance(point_1, point_2):
    "Given two points in (latitude, longitude) format, calculate the distance between them in miles"
    
    geod = Geodesic.WGS84


    g = geod.Inverse(point_1[0], point_1[1], point_2[0], point_2[1])
    miles = g['s12'] / 1000 * 0.621371
    
    return miles

In [60]:
fig = gmaps.figure(center=departure, zoom_level=9)

lines = []

lines.append(gmaps.Line(start=departure, end=arrival, stroke_color='blue'))

drawing_layer = gmaps.drawing_layer(features=lines)

departure_marker = gmaps.marker_layer([departure,arrival])
fig.add_layer(departure_marker)

fig.add_layer(drawing_layer)

print('Flying Distance: {:.2f}mi'.format(my_calculate_distance(departure,arrival)))
#Fly time is retrieved from calculations from previous query
print('Flying Time: 1.4min')

fig

Flying Distance: 1.05mi
Flying Time: 1.4min


Figure(layout=FigureLayout(height='420px'))